# Homework 7 (30 marks)
Create a copy of the notebook to start answering the questions. Name your notebook in the format HW7_lastname_firstname.ipynb to facilitate the grading process.

Answer all the questions, test your code to ensure there are no errors and the results are as expected. Once you have answered all the questions, save the final copy, then go to File-> click on Download.ipynb. Once the local copy has been downloaded, submit your file on Blackboard under the corresponding assignment section. Also provide us a link to your notebook during submission.

NOTE: Please give the TAs the permission to access your notebooks through the links you have provided during submission.

The due date of this homework is 04/23/2021 (Friday).

Please ensure you follow all the steps mentioned in the homework.

You can submit your solutions any number of times until the deadline.

The datasets used in this homework can be found in the google drive link below -

https://drive.google.com/drive/folders/1NxCh4X7u7wVo5aHojxjLNs9wC7B7zJhb?usp=sharing

Follow the necessary steps to import data to test your code. You can use any method to read your data in the notebook. We will not be grading the methods you use. We will only grade the code from the point where you read the dataset into a pandas dataframe - (pd.read_csv('file_name'))

Import all the libraries you require in the cell below.


In [71]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
# Enter your code here

## Implement the Ensemble methods learnt in class and compare their accuarcies.

The dataset you are going to be using for homework is the **Wisconsin Breast Cancer dataset (cancer.csv)**

The dataset contains a total number of 10 features labeled in either benign or malignant classes. The features have 699 instances out of which 16 feature values are missing. The dataset only contains numeric values.

Attribute Information:

1. Sample code number: id number
2. Clump Thickness: 1 - 10
3. Uniformity of Cell Size: 1 - 10
4. Uniformity of Cell Shape: 1 - 10
5. Marginal Adhesion: 1 - 10
6. Single Epithelial Cell Size: 1 - 10
7. Bare Nuclei: 1 - 10
8. Bland Chromatin: 1 - 10
9. Normal Nucleoli: 1 - 10
10. Mitoses: 1 - 10
11. Class: (2 for benign, 4 for malignant) (**target variable**)

For more information: https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(original)

### 1. Read the dataset into variable called '**data**' (1 mark)

In [16]:
pd.set_option('display.max_columns', 100)
# Enter your code here
data = pd.read_csv("cancer.csv")

In [17]:
data

,Sample Code Number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bland Chromatin,Bare Nuclei,Normal Nucleoli,Mitosis,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...
694,776715,3,1,1,1,3,2,1,1,1,2
695,841769,2,1,1,1,2,1,1,1,1,2
696,888820,5,10,10,3,7,3,8,10,2,4
697,897471,4,8,6,4,3,4,10,6,1,4


### **Preprocessing**: Data needs to be preprocessed before implementing ensemble methods. It is done for you here. 
### Run the below code first and then answer the questions from 2 - 7.

#### Deleting unnecessary columns: The column "Sample code number" is just an indicator and it's of no use in the modeling. So, let's drop it:


In [18]:
data.drop(['Sample Code Number'],axis = 1, inplace = True)


#### Handling missing values : 
As mentioned earlier, the dataset contains missing values. The column named "Bare Nuclei" contains them. The missing values are represneted as "?". 

Replace those "?"s with 0's and impute them with Mean Imputation

In [19]:
data['Bare Nuclei']

0       3
1       3
2       3
3       3
4       3
       ..
694     1
695     1
696     8
697    10
698    10
Name: Bare Nuclei, Length: 699, dtype: int64

In [20]:
data.replace('?',0, inplace=True)

In [22]:
# Convert the DataFrame object into NumPy array otherwise you will not be able to impute
values = data.values
# Now impute it
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputedData = imputer.fit_transform(values)

#### Normalizing the data:
Ranges of the features of the dataset are not the same. This may cause a problem. A small change in a feature might not affect the other. To address this problem, normalize the ranges of the features to a uniform range, in this case, 0 - 1.

In [15]:
scaler = MinMaxScaler(feature_range=(0, 1))
normalizedData = scaler.fit_transform(imputedData)
cols = ['Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape', 'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bland Chromatin', 'Bare Nuclei', 'Normal Nucleoli', 'Mitosis','Class']
normalizedData = pd.DataFrame(normalizedData, columns=cols)
print(normalizedData.head())

   Clump Thickness  Uniformity of Cell Size  Uniformity of Cell Shape  \
0         0.444444                 0.000000                  0.000000   
1         0.444444                 0.333333                  0.333333   
2         0.222222                 0.000000                  0.000000   
3         0.555556                 0.777778                  0.777778   
4         0.333333                 0.000000                  0.000000   

   Marginal Adhesion  Single Epithelial Cell Size  Bland Chromatin  \
0           0.000000                     0.111111              0.1   
1           0.444444                     0.666667              1.0   
2           0.000000                     0.111111              0.2   
3           0.000000                     0.222222              0.4   
4           0.222222                     0.111111              0.1   

   Bare Nuclei  Normal Nucleoli  Mitosis  Class  
0     0.222222         0.000000      0.0    0.0  
1     0.222222         0.111111      0.0

### Data preprocessing is done and now you will answer the below questions using the **normalizedData**: 

### 2. Split the data into test and training data with test size - 30%. Compute the baseline classification accuracy for X_train. (3 marks)

In [42]:
# Enter your code here

X = normalizedData.drop(['Class'], axis = 1)
y = normalizedData['Class']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3)

### 3.  Bagging : Build a generic Bagging ensemble and print the accuracy (4 marks)
---


Hyperparameters:

Base estimator = DecisionTreeClassifier

n_estimators = 10

random_state = 42

---


In [43]:
# Generic Bagging model
# Enter your code here
DTC = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=10, random_state=42).fit(X_train, y_train)
y_pred_dtc = DTC.predict(X_test)
print(y_pred_dtc)

[0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0.
 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1.
 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0.
 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0.
 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1.]


In [44]:
accuracy_dtc = accuracy_score(y_test, y_pred_dtc)
print(accuracy_dtc)

0.9523809523809523


### 4. RandomForest : (7 marks)
#### a) Build a Random Forest model and print the accuracy (4 marks)
---

Constructor arguments: 


n_estimators = 100, max_features = 7 and random_state = 42 


---




In [46]:
# Random Forest model
# Enter your code here
RFC = RandomForestClassifier(n_estimators=100, max_features = 7, random_state=42)
RFC.fit(X_train, y_train)
y_pred_rfc = RFC.predict(X_test)
accuracy_rfc = accuracy_score(y_test, y_pred_rfc)
print(accuracy_rfc)

0.9523809523809523


####  b) Calculate the top 3 important features for the above **RandomForest** model and print them (3 marks)

In [57]:
# Top 3 features for RandomForest
# Enter your code here
dic_rfc = dict(zip(X.columns, RFC.feature_importances_))
top_features_rfc = dict(sorted(dic_rfc.items(), key = lambda x: x[1], reverse=True))
print("all features", top_features_rfc)
count = 0
lst = []

for feature in top_features_rfc:
  count +=1
  if count <= 3:
    print(feature)
  else:
    break

all features {'Uniformity of Cell Size': 0.5954944361104588, 'Uniformity of Cell Shape': 0.16923957359518754, 'Bland Chromatin': 0.09321014542651306, 'Bare Nuclei': 0.03454890007636042, 'Clump Thickness': 0.030580786658503032, 'Normal Nucleoli': 0.029040691141025296, 'Single Epithelial Cell Size': 0.026211185668461742, 'Marginal Adhesion': 0.01632467400574144, 'Mitosis': 0.0053496073177484805}
Uniformity of Cell Size
Uniformity of Cell Shape
Bland Chromatin


### 5. Boosting: (7 marks)
#### a) Build an AdaBoost model with training data and print the accuracy (4 marks)
---

Hyperparameters:

Base estimator = DecisionTreeClassifier, max_depth = 4

n_estimators = 200

random_state = 42

learning_rate = 0.05


---









In [56]:
# AdaBoost Classification
# Enter your code here
ABC = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth=4), learning_rate = 0.05, n_estimators=200, random_state=42)
ABC.fit(X_train, y_train)
y_pred_abc = ABC.predict(X_test)
accuracy_abc = accuracy_score(y_test, y_pred_abc)
print(accuracy_abc)

0.9571428571428572


#### b) Calculate the top 3 important features for the above **AdaBoost** model and print them (3 marks)

In [58]:
# Top 3 features for AdaBoost
# Enter your code here
dic_abc = dict(zip(X.columns, ABC.feature_importances_))
top_features_abc = dict(sorted(dic_abc.items(), key = lambda x: x[1], reverse=True))
print("all features", top_features_abc)
count = 0
lst = []

for feature in top_features_abc:
  count +=1
  if count <= 3:
    print(feature)
  else:
    break


all features {'Marginal Adhesion': 0.42940818683012155, 'Normal Nucleoli': 0.19073313849764134, 'Clump Thickness': 0.11159092670826572, 'Single Epithelial Cell Size': 0.09514988457382374, 'Uniformity of Cell Shape': 0.09412199533305214, 'Bland Chromatin': 0.038542694965307, 'Uniformity of Cell Size': 0.029891170760840757, 'Bare Nuclei': 0.006417733248956835, 'Mitosis': 0.004144269081990828}
Marginal Adhesion
Normal Nucleoli
Clump Thickness


### 6. Voting : Using a voting classifier, build an ensemble of RandomForestClassifier, DecisionTreeClassifier, Support Vector Machine and Logistic Regression. (7 marks)


---


Use max_depth = 4, n_estimators = 200, voting = soft

In [64]:
# Voting Ensemble for Classification
# Enter your code here
rfc = RandomForestClassifier(n_estimators=200,max_depth=4)
dtc = DecisionTreeClassifier(max_depth = 4)
svc = SVC(probability=True)
lr = LogisticRegression()
VC = VotingClassifier(estimators=[("rfc",rfc),("dtc",dtc),("svc",svc),("lr",lr)], voting ='soft')
VC = VC.fit(X_train, y_train)
y_pred_vc = VC.predict(X_test)
accuracy_vc = accuracy_score(y_test, y_pred_vc)
print(accuracy_vc)

0.9571428571428572


### 7. Mention the best model among the above 4 models and its accuracy (1 mark)

In [73]:
for clf, label in zip([rfc, dtc, svc, lr, VC], ['Random Forest', 'Decision Tree', 'Support Vector Machines', 'Logistic Regression','Voting Classifer']):
  scores = cross_val_score(clf, X, y, scoring='accuracy', cv=5)
  print("Accuracy: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.9671 (+/- 0.0200) [Random Forest]
Accuracy: 0.9327 (+/- 0.0243) [Decision Tree]
Accuracy: 0.9643 (+/- 0.0271) [Support Vector Machines]
Accuracy: 0.9600 (+/- 0.0308) [Logistic Regression]
Accuracy: 0.9614 (+/- 0.0288) [Voting Classifer]


The best model among the 4 is Random Forest with accuracy at .9671.